# 대화형 검색 - Conversational Search


In [8]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install -q langchain
!pip install -q langchain_community

## Amazon Bedrock Claude Sonnet 사용하기


In [9]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import boto3
import json
import textwrap

In [10]:
query_text = "영화 건축학개론 줄거리와 평점은?"

In [11]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage

region_name = "us-east-1"

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
}

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
    region_name=region_name,
)  # Claude 속성 구성

messages = [HumanMessage(content=query_text)]

print(textwrap.fill(llm(messages).content, 80))

/opt/anaconda3/envs/opensearch/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(
/opt/anaconda3/envs/opensearch/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


영화 '건축학개론'은 2012년 개봉한 한국 영화입니다.   줄거리: 이 영화는 건축가를 꿈꾸는 대학생 서남준의 이야기를 그리고 있습니다.
남준은 건축학과에 입학하지만 현실과 이상 사이에서 갈등을 겪게 됩니다. 그는 자신만의 건축 철학을 찾아가는 과정에서 사랑, 우정, 꿈 등 인생의
여러 모습을 경험하게 됩니다. 건축에 대한 열정과 고민을 섬세하게 다룬 작품입니다.  평점: - 네이버 영화 평점: 7.82/10 (참여 평가
2,500여 명) - 다음 영화 관람객 평점: 3.9/5 (참여 평가 1,000여 명)  전반적으로 긍정적인 평가를 받았습니다. 건축에 대한
열정과 고민을 잘 표현했다는 호평이 많았지만, 다소 지루하다는 지적도 있었습니다. 하지만 독특한 소재와 섬세한 연출력을 인정받은 작품으로
평가되고 있습니다.


## RAG을 통해 환각 없애기


### Lexical Search 이용하기


In [12]:
# Add system path for utils
import sys

sys.path.insert(0, "/Users/jinhwan/Repository/Labs/opensearch-with-bedrock-workshop-kr/utils")

In [13]:
import boto3, json
from utils import get_cfn_outputs

region_name = "us-east-1"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

In [14]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

In [15]:
import numpy as np
from langchain_community.embeddings import BedrockEmbeddings
from requests_aws4auth import AWS4Auth
from typing import Any, Dict, Iterable, List, Optional, Tuple, Callable
from typing import Type

aos_url = "https://" + aos_host

index_name = "movie_semantic"

embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name=region_name)

# this is just an example, you would need to change these values to point to another opensearch instance
docsearch = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embeddings,
    opensearch_url=aos_url,
    http_auth=auth,
    connection_class=RequestsHttpConnection,
)


class SimiliarOpenSearchVectorSearch(OpenSearchVectorSearch):

    def relevance_score(self, distance: float) -> float:
        return distance

    def _select_relevance_score_fn(self) -> Callable[[float], float]:
        return self.relevance_score


open_search_vector_store = SimiliarOpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embeddings,
    opensearch_url=aos_url,
    http_auth=auth,
    connection_class=RequestsHttpConnection,
)

In [16]:
from langchain.schema import Document


def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        metadata = {"score": hit["_score"], "id": hit["_id"]}

        content = {
            "title": hit["_source"]["title"],
            "genre": hit["_source"]["genre"],
            "rating": hit["_source"]["rating"],
            "text": hit["_source"]["text"],
        }

        doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)

        query_result.append(doc)

    return query_result

In [17]:
# you can specify custom field names to match the fields you're using to store your embedding, document text value, and metadata
# docs = open_search_vector_store.similarity_search_with_score(
#     query_text,
#     search_type="approximate_search",
#     space_type="cosinesimil",
#     k=10,
# )

# docs_ = open_search_vector_store.similarity_search_with_score(question, k=5)

docs = keyword_search(query_text)

print("found document number:" + str(len(docs)))

print("opensearch results:\n")
for doc in docs:
    print(doc)
    print("\n-----------------")

found document number:10
opensearch results:

page_content='{"title": "건축학개론", "genre": "멜로/로맨스", "rating": 8.67, "text": "생기 넘치지만 숫기 없던 스무 살, 건축학과 승민은 \'건축학개론\' 수업에서 처음 만난 음대생 서연에게 반한다. 함께 숙제를 하게 되면서 차츰 마음을 열고 친해지지만, 자신의 마음을 표현하는 데 서툰 순진한 승민은 입 밖에 낼 수 없었던 고백을 마음 속에 품은 채 작은 오해로 인해 서연과 멀어지게 된다. 어쩌면 다시…사랑할 수 있을까? 15년 만에 그녀를 다시 만났다 서른 다섯의 건축가가 된 승민 앞에 15년 만에 불쑥 나타난 서연. 당황스러움을 감추지 못하는 승민에게 서연은 자신을 위한 집을 설계해달라고 한다. 자신의 이름을 건 첫 작품으로 서연의 집을 짓게 된 승민, 함께 집을 완성해 가는 동안 어쩌면 사랑이었을지 모를 그때의 기억이 되살아나 두 사람 사이에 새로운 감정이 쌓이기 시작하는데…"}' metadata={'score': 9.500956, 'id': '5bAQ1I8BhWoFUAg8sGxr'}

-----------------
page_content='{"title": "무서운 영화", "genre": "코미디|공포|미스터리", "rating": 7.61, "text": "{\\"나는 네가 지난 할로윈 데이에 소리치는 걸보고 육감적으로 뭔가 특별한 일이 일어날 것이라는 것을 알았다!!\\"} 팝콘을 튀기던 드류에게 음산한 목소리의 남자에게서 이상한 전화가 걸려온다. \\"공포영화를 좋아하냐?\\"는 괴기한 질문을 던지고, 갑자기 등장해 그녀를 해치려 든다. 달아나는 드류는 속옷바람으로 슈퍼모델인양 멋진 포즈로 취하기도 하지만 결국 집으로 오던 그녀의 아버지의 차에 치어 죽고 만다. 드류가 죽었다는 소식을 전해들은 친구 신디는 드류의 죽음에 어떤 의문이 숨어있을까를 고민하다 문득 지난 할로윈 데이에 있었던 일을 떠올리는데.. 흥겨운 

검색 비교


In [18]:
from langchain import PromptTemplate

prompt_template = """


Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [19]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=PROMPT, verbose=True)

answer = chain.run(input_documents=docs, question=query_text)

print("##############################")
print("query: ", query_text)
answer_str = "answer: \n" + answer
print(textwrap.fill(answer_str, 80))

/opt/anaconda3/envs/opensearch/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.

<context>
{"title": "건축학개론", "genre": "멜로/로맨스", "rating": 8.67, "text": "생기 넘치지만 숫기 없던 스무 살, 건축학과 승민은 '건축학개론' 수업에서 처음 만난 음대생 서연에게 반한다. 함께 숙제를 하게 되면서 차츰 마음을 열고 친해지지만, 자신의 마음을 표현하는 데 서툰 순진한 승민은 입 밖에 낼 수 없었던 고백을 마음 속에 품은 채 작은 오해로 인해 서연과 멀어지게 된다. 어쩌면 다시…사랑할 수 있을까? 15년 만에 그녀를 다시 만났다 서른 다섯의 건축가가 된 승민 앞에 15년 만에 불쑥 나타난 서연. 당황스러움을 감추지 못하는 승민에게 서연은 자신을 위한 집을 설계해달라고 한다. 자신의 이름을 건 첫 작품으로 서연의 집을 짓게 된 승민, 함께 집을 완성해 가는 동안 어쩌면 사랑이었을지 모를 그때의 기억이 되살아나 두 사람 사이에 새로운 감정이 쌓이기 시작하는데…"}

{"title": "무서운 영화", "genre": "코미디|공포|미스터리", "rating": 7.61, "text": "{\"나는 네가 지난 할로윈 데이에 소리치는 걸보고 육감적으로 뭔가 특별한 일이 일어날 것이라는 것을 알았다!!\"} 팝콘을 튀기던 드류에게 음산한 목소리의 남자에게서 이상한 전화가 걸려온다. \"공포영화를 좋아하냐?\"는 괴기한 질문을 던지고, 갑자기 등장해 그녀를 해치려 든다. 달아나는 드류는 속옷바람으로 슈퍼모델인양 멋진 포즈로 취하기도 하지만 결국 집으로 오던 그녀의 아버지의 차에 치어 죽고 만다. 드류가 죽었다는 소식을 전해들은 친구 신디는 드류의 죽음에 어떤 의문이 숨어있을까를 고민하다 문득 지난 

### 하이브리드 검색 이용하기


In [20]:
def hybrid_search(query_text):
    query = {
        "size": 10,
        "_source": {"exclude": ["vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title", "text", "genre"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": "jK-l048BhWoFUAg8WOdO",
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [0.3, 0.7]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        metadata = {"score": hit["_score"], "id": hit["_id"]}

        content = {
            "title": hit["_source"]["title"],
            "genre": hit["_source"]["genre"],
            "rating": hit["_source"]["rating"],
            "text": hit["_source"]["text"],
            "score": hit["_score"],
        }

        doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)
        query_result.append(doc)

    return query_result

In [21]:
prompt_template = """


Human: Here is the list of recommended movies, inside <movies></movies> XML tags.

<movies>
{context}
</movies>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

You are a best movie reviewer in Korea. Please recommend a movies from the list above.

Question:
{question}

If the answer is not in the context, just say "추천해드릴만한 영화가 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [22]:
query_text = "우주에서 벌어지는 전쟁이야기"

docs = hybrid_search(query_text)
print("opensearch results:\n")
for doc in docs:
    print(doc)
    print("\n-----------------")

opensearch results:

page_content='{"title": "하이 라이프", "genre": "SF|스릴러", "rating": 6.56, "text": "태양계 너머 우주 공간에서 실험 대상이 되기로 받아들인 한 범죄자 무리는 우주선 내에서 모종의 실험을 하고 있다. 어느 날 이들은 믿을 수 없는 진실과 마주하게 되면서 혼란에 빠지게 되는데…", "score": 0.7}' metadata={'score': 0.7, 'id': 'JLAP1I8BhWoFUAg8tldM'}

-----------------
page_content='{"title": "스페이스 둠스데이", "genre": "판타지|SF", "rating": 6.2, "text": "미래 282 년, 우주를 지배하려는 외계 종족의 위협에 맞서 새로운 삶의 터전을 찾기 위해 고군분투하는 인간들의 이야기를 그린 내용", "score": 0.69572365}' metadata={'score': 0.69572365, 'id': 'O7AQ1I8BhWoFUAg8P2Pz'}

-----------------
page_content='{"title": "인디펜던스 데이: 리써전스", "genre": "액션|모험|SF", "rating": 7.25, "text": "다시, 그들이 온다! 2 년 전 최악의 우주 전쟁을 치른 지구. 재건을 위해 힘쓴 전세계는 다시 한번 있을 외계의 침공에 대비한다. 반드시 살아남아야 한다. 마침내 돌아온 그날! 상상을 초월하는 그들의 공격에 앞에 인류 최후의 전쟁이 시작된다.", "score": 0.5734711}' metadata={'score': 0.5734711, 'id': 'mbAQ1I8BhWoFUAg8IWBv'}

-----------------
page_content='{"title": "감자별 2013QR3", "genre": "드라마", "rating": 9.49, "text": "2 13년 어느 날 지구로 날아온 의문의 행성 \'감자별\' 때문에 벌

In [23]:
chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=PROMPT, verbose=True)

answer = chain.run(input_documents=docs, question=query_text)

print("##############################")
print("query: ", query_text)
answer_str = "answer: \n" + answer
print(textwrap.fill(answer_str, 80))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the list of recommended movies, inside <movies></movies> XML tags.

<movies>
{"title": "하이 라이프", "genre": "SF|스릴러", "rating": 6.56, "text": "태양계 너머 우주 공간에서 실험 대상이 되기로 받아들인 한 범죄자 무리는 우주선 내에서 모종의 실험을 하고 있다. 어느 날 이들은 믿을 수 없는 진실과 마주하게 되면서 혼란에 빠지게 되는데…", "score": 0.7}

{"title": "스페이스 둠스데이", "genre": "판타지|SF", "rating": 6.2, "text": "미래 282 년, 우주를 지배하려는 외계 종족의 위협에 맞서 새로운 삶의 터전을 찾기 위해 고군분투하는 인간들의 이야기를 그린 내용", "score": 0.69572365}

{"title": "인디펜던스 데이: 리써전스", "genre": "액션|모험|SF", "rating": 7.25, "text": "다시, 그들이 온다! 2 년 전 최악의 우주 전쟁을 치른 지구. 재건을 위해 힘쓴 전세계는 다시 한번 있을 외계의 침공에 대비한다. 반드시 살아남아야 한다. 마침내 돌아온 그날! 상상을 초월하는 그들의 공격에 앞에 인류 최후의 전쟁이 시작된다.", "score": 0.5734711}

{"title": "감자별 2013QR3", "genre": "드라마", "rating": 9.49, "text": "2 13년 어느 날 지구로 날아온 의문의 행성 '감자별' 때문에 벌어지는 노씨 일가의 좌충우돌 멘붕 스토리를 담은 일일시트콤", "score": 0.5014462}

{"title": "스페셜 솔져", "genre": "액션|전쟁", "rating": 6.8, "

## 대화형 검색 구현


In [24]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=10, return_messages=True)

In [42]:
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import BaseRetriever
from typing import Any, List
from langchain.schema import Document


class OpenSearchLexicalSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    k = 10
    minimum_should_match = 0
    filter = []

    def _reset_search_params(
        self,
    ):

        self.k = 10
        self.minimum_should_match = 0
        self.filter = []

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        body = {
            "size": self.k,
            "_source": {"exclude": ["vector_field"]},
            "query": {
                "hybrid": {
                    "queries": [
                        {
                            "multi_match": {
                                "query": query_text,
                                "fields": ["title", "text", "genre"],
                            }
                        },
                        {
                            "neural": {
                                "vector_field": {
                                    "query_text": query_text,
                                    "model_id": "jK-l048BhWoFUAg8WOdO",
                                    "k": 30,
                                }
                            }
                        },
                    ]
                }
            },
            "search_pipeline": {
                "description": "Post processor for hybrid search",
                "phase_results_processors": [
                    {
                        "normalization-processor": {
                            "normalization": {"technique": "min_max"},
                            "combination": {
                                "technique": "arithmetic_mean",
                                "parameters": {"weights": [0.3, 0.7]},
                            },
                        }
                    }
                ],
            },
        }
        res = self.os_client.search(index=index_name, body=body)

        query_result = []

        for hit in res["hits"]["hits"]:
            metadata = {"score": hit["_score"], "id": hit["_id"]}

            content = {
                "title": hit["_source"]["title"],
                "genre": hit["_source"]["genre"],
                "rating": hit["_source"]["rating"],
                "text": hit["_source"]["text"],
                "score": hit["_score"],
            }

            doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)
            query_result.append(doc)

        return query_result

In [43]:
hybrid_retriever = OpenSearchLexicalSearchRetriever(os_client=aos_client, index_name=index_name)

search_hybrid_result = hybrid_retriever.get_relevant_documents(query_text)

answer = chain.run(input_documents=search_hybrid_result, question=query_text)

print("##############################")
print("query: \n", query_text)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the list of recommended movies, inside <movies></movies> XML tags.

<movies>
{"title": "하이 라이프", "genre": "SF|스릴러", "rating": 6.56, "text": "태양계 너머 우주 공간에서 실험 대상이 되기로 받아들인 한 범죄자 무리는 우주선 내에서 모종의 실험을 하고 있다. 어느 날 이들은 믿을 수 없는 진실과 마주하게 되면서 혼란에 빠지게 되는데…", "score": 0.7}

{"title": "스페이스 둠스데이", "genre": "판타지|SF", "rating": 6.2, "text": "미래 282 년, 우주를 지배하려는 외계 종족의 위협에 맞서 새로운 삶의 터전을 찾기 위해 고군분투하는 인간들의 이야기를 그린 내용", "score": 0.69572365}

{"title": "인디펜던스 데이: 리써전스", "genre": "액션|모험|SF", "rating": 7.25, "text": "다시, 그들이 온다! 2 년 전 최악의 우주 전쟁을 치른 지구. 재건을 위해 힘쓴 전세계는 다시 한번 있을 외계의 침공에 대비한다. 반드시 살아남아야 한다. 마침내 돌아온 그날! 상상을 초월하는 그들의 공격에 앞에 인류 최후의 전쟁이 시작된다.", "score": 0.5734711}

{"title": "감자별 2013QR3", "genre": "드라마", "rating": 9.49, "text": "2 13년 어느 날 지구로 날아온 의문의 행성 '감자별' 때문에 벌어지는 노씨 일가의 좌충우돌 멘붕 스토리를 담은 일일시트콤", "score": 0.5014462}

{"title": "스페셜 솔져", "genre": "액션|전쟁", "rating": 6.8, "

In [44]:
condense_template = """
Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>
{chat_history}
</chat-history>

<follow-up-question>
{question}
</follow-up-question>

standalone question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

In [45]:
from langchain.chains import ConversationalRetrievalChain

memory.clear()

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=hybrid_retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    verbose=True,
)

chat_response = conversation_with_retrieval.invoke({"question": query_text})

print(textwrap.fill(chat_response["answer"], 80))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the list of recommended movies, inside <movies></movies> XML tags.

<movies>
{"title": "하이 라이프", "genre": "SF|스릴러", "rating": 6.56, "text": "태양계 너머 우주 공간에서 실험 대상이 되기로 받아들인 한 범죄자 무리는 우주선 내에서 모종의 실험을 하고 있다. 어느 날 이들은 믿을 수 없는 진실과 마주하게 되면서 혼란에 빠지게 되는데…", "score": 0.7}

{"title": "스페이스 둠스데이", "genre": "판타지|SF", "rating": 6.2, "text": "미래 282 년, 우주를 지배하려는 외계 종족의 위협에 맞서 새로운 삶의 터전을 찾기 위해 고군분투하는 인간들의 이야기를 그린 내용", "score": 0.69572365}

{"title": "인디펜던스 데이: 리써전스", "genre": "액션|모험|SF", "rating": 7.25, "text": "다시, 그들이 온다! 2 년 전 최악의 우주 전쟁을 치른 지구. 재건을 위해 힘쓴 전세계는 다시 한번 있을 외계의 침공에 대비한다. 반드시 살아남아야 한다. 마침내 돌아온 그날! 상상을 초월하는 그들의 공격에 앞에 인류 최후의 전쟁이 시작된다.", "score": 0.5734711}

{"title": "감자별 2013QR3", "genre": "드라마", "rating": 9.49, "text": "2 13년 어느 날 지구로 날아온 의문의 행성 '감자별' 때문에 벌어지는 노씨 일가의 좌충우돌 멘붕 스토리를 담은 일일시트콤", "score": 0.5014462}

{"title": "스페셜 솔져", "genre": "액션|전쟁", "rating": 6.8, "

In [46]:
chat_response = conversation_with_retrieval.invoke({"question": "그 영화 평점은?"})

print(textwrap.fill(chat_response["answer"], 80))



> Entering new LLMChain chain...
Prompt after formatting:

Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>

Human: 우주에서 벌어지는 전쟁이야기
Assistant: 제가 추천해드릴 수 있는 우주에서 벌어지는 전쟁 이야기 영화는 다음과 같습니다.

1. 인디펜던스 데이: 리써전스 (Independence Day: Resurgence)
- 장르: 액션, 모험, SF
- 평점: 7.25
- 줄거리: 2년 전 최악의 우주 전쟁을 치른 지구가 다시 한번 외계의 침공에 맞서 인류 최후의 전쟁을 펼치는 이야기입니다.

2. 스페셜 솔져 (Special Soldier)  
- 장르: 액션, 전쟁
- 평점: 6.8
- 줄거리: 세계 곳곳에서 전쟁이 동시다발적으로 터지면서 특수부대가 출동하여 현재와 미래를 건 전쟁에 맞서는 내용입니다.

두 영화 모두 우주와 지구를 배경으로 전쟁 이야기를 그리고 있어 흥미롭게 보실 수 있을 것 같습니다.
</chat-history>

<follow-up-question>
그 영화 평점은?
</follow-up-question>

standalone question:


> Finished chain.


> Entering new StuffDocuments

In [47]:
chat_response = conversation_with_retrieval.invoke({"question": "비슷한 장르의 다른 영화는?"})

print(textwrap.fill(chat_response["answer"], 80))



> Entering new LLMChain chain...
Prompt after formatting:

Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>

Human: 우주에서 벌어지는 전쟁이야기
Assistant: 제가 추천해드릴 수 있는 우주에서 벌어지는 전쟁 이야기 영화는 다음과 같습니다.

1. 인디펜던스 데이: 리써전스 (Independence Day: Resurgence)
- 장르: 액션, 모험, SF
- 평점: 7.25
- 줄거리: 2년 전 최악의 우주 전쟁을 치른 지구가 다시 한번 외계의 침공에 맞서 인류 최후의 전쟁을 펼치는 이야기입니다.

2. 스페셜 솔져 (Special Soldier)  
- 장르: 액션, 전쟁
- 평점: 6.8
- 줄거리: 세계 곳곳에서 전쟁이 동시다발적으로 터지면서 특수부대가 출동하여 현재와 미래를 건 전쟁에 맞서는 내용입니다.

두 영화 모두 우주와 지구를 배경으로 전쟁 이야기를 그리고 있어 흥미롭게 보실 수 있을 것 같습니다.
Human: 그 영화 평점은?
Assistant: '인디펜던스 데이: 리써전스'의 평점은 7.25, '스페셜 솔져'의 평점은 6.8입니다. 두 영화 모두 우주에서 벌어지는 전쟁 이야기를 다루고 있습니다.
</chat-history>

<follow-up-q

## Clean Up


In [35]:
# aos_client.indices.delete(index=index_name)
memory.clear()